# Problem Set 5: Empirical Problem - Duranton and Turner
## Urban Economics, Fall 2025

In this problem set, we will explore the Duranton and Turner paper using Python. This notebook contains the empirical questions that require coding solutions.

## Setup and Data Loading

**Variable Descriptions:**
- `vmt_IH_YR`: Mean daily VKT (Vehicle Kilometers Traveled) for year YR (interstates)
- `ln_km_IH_YR`: Mean lane km for year YR (interstates)
- `popYR`: Population for year YR (in MSA)

**Important:** The authors exclude from these analyses any MSA for which they do not have data on km of highway in 1983, 1993, or 2003. A missing data point for the mean lane km variable is one where the data table has a zero for that MSA for that year.

**Task:** Load the necessary libraries and the dataset. Filter the data appropriately.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.sandwich_covariance import cov_hc1

# Load the parquet file (converted from dtData-1.RData)
df = pd.read_parquet('dtData-1.parquet')
print(f"Original dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

Original dataset shape: (275, 213)
Columns: ['msa', 'msa_name', 'l_max_84bus', 'l_max_94bus', 'l_max_04bus', 'l_transit84', 'l_transit94', 'l_transit04', 'l_pop20', 'l_pop30', 'l_pop40', 'l_pop50', 'l_pop60', 'l_pop70', 'l_pop80', 'l_pop90', 'l_pop00', 'D_lpop00', 'D_lpop90', 'D_lpop80', 'D_lpop', 'pc_aquifer_msa', 'elevat_range_msa', 'ruggedness_msa', 'cooling_dd', 'heating_dd', 'seg1980_ghetto', 'pc_incorp_msa', 'pc_aquifer_msa2', 'elevat_range_msa2', 'ruggedness_msa2', 'eleva_rug', 'division', 'div1', 'div2', 'div3', 'div4', 'div5', 'div6', 'div7', 'div8', 'div9', 'sprawl_1992', 'sprawl_1976', 'S_somecollege_80', 'S_somecollege_00', 'S_somecollege_90', 'S_poor_80', 'S_poor_00', 'S_poor_90', 'l_mean_income_80', 'l_mean_income_90', 'l_mean_income_00', 'S_manuf83', 'S_manuf93', 'S_manuf03', 'S_truck83', 'S_truck93', 'S_truck03', 'S_wholet83', 'S_wholet93', 'S_wholet03', 'l_hwy1947', 'l_rail1898', 'demvote_sh72', 'max_04bus', 'max_94bus', 'max_84bus', 'emp80', 'emp83', 'emp90', 'emp93',

In [2]:
# Data filtering (equivalent to R's subset function)
# Remove observations where ln_km_IH variables are missing or zero for any of the three years
dtDataUse = df[
    (df['ln_km_IH_83'].notna()) & 
    (df['ln_km_IH_93'].notna()) & 
    (df['ln_km_IH_03'].notna()) & 
    (df['ln_km_IH_83'] > 0) & 
    (df['ln_km_IH_93'] > 0) & 
    (df['ln_km_IH_03'] > 0)
].copy()

# Select relevant variables
variables_to_keep = ['vmt_IH_83', 'vmt_IH_93', 'vmt_IH_03', 
                     'ln_km_IH_83', 'ln_km_IH_93', 'ln_km_IH_03', 
                     'pop80', 'pop90', 'pop00']
dtDataUse = dtDataUse[variables_to_keep]

print(f"Filtered dataset shape: {dtDataUse.shape}")
print(f"Number of observations removed: {df.shape[0] - dtDataUse.shape[0]}")

Filtered dataset shape: (228, 9)
Number of observations removed: 47


## Question 2.1: Table 1 - Summary Statistics

**Task:** Generate a summary table with the **average across MSAs** for the three main variables (VMT, lane km, population) for all years. You do not need standard deviations.

In [3]:
# Question 1: Summary statistics (means across MSAs)
# Calculate means for all variables (equivalent to R's lapply(dtDataUse, mean, na.rm=TRUE))
summary_stats = dtDataUse.mean()
print("Average values across MSAs:")
print("="*50)
for var, mean_val in summary_stats.items():
    print(f"{var}: {mean_val:.2f}")

# Also display full summary (equivalent to R's summary())
print("\nDetailed summary statistics:")
print("="*50)
print(dtDataUse.describe())

Average values across MSAs:
vmt_IH_83: 7776633.69
vmt_IH_93: 11904953.05
vmt_IH_03: 15960581.06
ln_km_IH_83: 1140.27
ln_km_IH_93: 1208.16
ln_km_IH_03: 1279.75
pop80: 753726.62
pop90: 834290.29
pop00: 950054.31

Detailed summary statistics:
          vmt_IH_83     vmt_IH_93     vmt_IH_03   ln_km_IH_83   ln_km_IH_93  \
count  2.280000e+02  2.280000e+02  2.280000e+02    228.000000    228.000000   
mean   7.776634e+06  1.190495e+07  1.596058e+07   1140.266380   1208.157797   
std    1.662398e+07  2.425106e+07  3.157929e+07   1649.762170   1729.431814   
min    7.131325e+03  4.264949e+05  6.688282e+05      8.304215    111.344917   
25%    1.083210e+06  1.715880e+06  2.406337e+06    361.333082    345.923853   
50%    2.195813e+06  3.518657e+06  4.938078e+06    568.804287    604.655202   
75%    6.536746e+06  1.095113e+07  1.558873e+07   1166.674781   1256.166261   
max    1.345363e+08  2.133462e+08  2.710781e+08  12611.230499  13628.848404   

        ln_km_IH_03           pop80           po

## Question 2.2: Table 3 - Cross-Sectional Regressions

**Task:** Replicate the results from columns (1), (4), (7) of Table 3. These are cross-sectional regressions for 1983, 1993, and 2003.

**Regression specification:** 
```
log(vmt_IH_YR) = β₀ + β₁*log(ln_km_IH_YR) + β₂*log(popYR) + ε
```

**Note:** Use heteroskedasticity-robust standard errors (HC1 type).

In [9]:
# 1983 Cross-sectional regression: log(vmt_IH_83) ~ log(ln_km_IH_83) + log(pop80)
# Create log-transformed variables
dtDataUse['log_vmt_83'] = np.log(dtDataUse['vmt_IH_83'])
dtDataUse['log_km_83'] = np.log(dtDataUse['ln_km_IH_83'])
dtDataUse['log_pop80'] = np.log(dtDataUse['pop80'])

# Run OLS regression
ols83 = smf.ols('log_vmt_83 ~ log_km_83 + log_pop80', data=dtDataUse).fit()

# Get robust standard errors using get_robustcov_results() method
ols83_robust = ols83.get_robustcov_results(cov_type='HC1')

print("1983 Regression Results (with robust standard errors):")
print("="*60)
print(f"{'Variable':<15} {'Coefficient':<12} {'Robust SE':<12} {'t-stat':<12}")
print("-" * 60)

# Use the original parameter names and access robust results by index
for i, param_name in enumerate(ols83.params.index):
    coef = ols83_robust.params[i]
    se = ols83_robust.bse[i]
    t_stat = ols83_robust.tvalues[i]
    print(f"{param_name:<15} {coef:>11.4f} {se:>11.4f} {t_stat:>11.4f}")

print(f"\nR-squared: {ols83_robust.rsquared:.4f}")
print(f"Observations: {ols83_robust.nobs}")

# Key coefficient (elasticity of VKT w.r.t. highway km)
# log_km_83 is the second parameter (index 1)
elasticity_83 = ols83_robust.params[1]
print(f"\n*** 1983 VKT elasticity w.r.t. highway km: {elasticity_83:.4f} ***")

1983 Regression Results (with robust standard errors):
Variable        Coefficient  Robust SE    t-stat      
------------------------------------------------------------
Intercept            3.3048      0.3042     10.8650
log_km_83            0.9162      0.0564     16.2384
log_pop80            0.4382      0.0420     10.4289

R-squared: 0.9273
Observations: 228.0

*** 1983 VKT elasticity w.r.t. highway km: 0.9162 ***


In [11]:
# 1993 Cross-sectional regression: log(vmt_IH_93) ~ log(ln_km_IH_93) + log(pop90)
# Create log-transformed variables
dtDataUse['log_vmt_93'] = np.log(dtDataUse['vmt_IH_93'])
dtDataUse['log_km_93'] = np.log(dtDataUse['ln_km_IH_93'])
dtDataUse['log_pop90'] = np.log(dtDataUse['pop90'])

# Run OLS regression
ols93 = smf.ols('log_vmt_93 ~ log_km_93 + log_pop90', data=dtDataUse).fit()

# Get robust standard errors using get_robustcov_results() method
ols93_robust = ols93.get_robustcov_results(cov_type='HC1')

print("1993 Regression Results (with robust standard errors):")
print("="*60)
print(f"{'Variable':<15} {'Coefficient':<12} {'Robust SE':<12} {'t-stat':<12}")
print("-" * 60)

# Use the original parameter names and access robust results by index
for i, param_name in enumerate(ols93.params.index):
    coef = ols93_robust.params[i]
    se = ols93_robust.bse[i]
    t_stat = ols93_robust.tvalues[i]
    print(f"{param_name:<15} {coef:>11.4f} {se:>11.4f} {t_stat:>11.4f}")

print(f"\nR-squared: {ols93_robust.rsquared:.4f}")
print(f"Observations: {ols93_robust.nobs}")

# Key coefficient (elasticity of VKT w.r.t. highway km)
elasticity_93 = ols93_robust.params[1]  # Second parameter (index 1)
print(f"\n*** 1993 VKT elasticity w.r.t. highway km: {elasticity_93:.4f} ***")

1993 Regression Results (with robust standard errors):
Variable        Coefficient  Robust SE    t-stat      
------------------------------------------------------------
Intercept            3.5802      0.3199     11.1902
log_km_93            0.7268      0.0449     16.1786
log_pop90            0.5443      0.0422     12.9010

R-squared: 0.9370
Observations: 228.0

*** 1993 VKT elasticity w.r.t. highway km: 0.7268 ***


In [12]:
# 2003 Cross-sectional regression: log(vmt_IH_03) ~ log(ln_km_IH_03) + log(pop00)
# Create log-transformed variables
dtDataUse['log_vmt_03'] = np.log(dtDataUse['vmt_IH_03'])
dtDataUse['log_km_03'] = np.log(dtDataUse['ln_km_IH_03'])
dtDataUse['log_pop00'] = np.log(dtDataUse['pop00'])

# Run OLS regression
ols03 = smf.ols('log_vmt_03 ~ log_km_03 + log_pop00', data=dtDataUse).fit()

# Get robust standard errors using get_robustcov_results() method
ols03_robust = ols03.get_robustcov_results(cov_type='HC1')

print("2003 Regression Results (with robust standard errors):")
print("="*60)
print(f"{'Variable':<15} {'Coefficient':<12} {'Robust SE':<12} {'t-stat':<12}")
print("-" * 60)

# Use the original parameter names and access robust results by index
for i, param_name in enumerate(ols03.params.index):
    coef = ols03_robust.params[i]
    se = ols03_robust.bse[i]
    t_stat = ols03_robust.tvalues[i]
    print(f"{param_name:<15} {coef:>11.4f} {se:>11.4f} {t_stat:>11.4f}")

print(f"\nR-squared: {ols03_robust.rsquared:.4f}")
print(f"Observations: {ols03_robust.nobs}")

# Key coefficient (elasticity of VKT w.r.t. highway km)
elasticity_03 = ols03_robust.params[1]  # Second parameter (index 1)
print(f"\n*** 2003 VKT elasticity w.r.t. highway km: {elasticity_03:.4f} ***")

# Summary of elasticities over time
print(f"\n{'='*50}")
print("SUMMARY: VKT Elasticity w.r.t. Highway KM Over Time")
print(f"{'='*50}")
print(f"1983: {elasticity_83:.4f}")
print(f"1993: {elasticity_93:.4f}")  
print(f"2003: {elasticity_03:.4f}")
print(f"Change 1983-2003: {elasticity_03 - elasticity_83:.4f}")
print("Trend: Elasticity is decreasing over time")

2003 Regression Results (with robust standard errors):
Variable        Coefficient  Robust SE    t-stat      
------------------------------------------------------------
Intercept            4.0908      0.3273     12.4988
log_km_03            0.7080      0.0457     15.4916
log_pop00            0.5319      0.0437     12.1801

R-squared: 0.9407
Observations: 228.0

*** 2003 VKT elasticity w.r.t. highway km: 0.7080 ***

SUMMARY: VKT Elasticity w.r.t. Highway KM Over Time
1983: 0.9162
1993: 0.7268
2003: 0.7080
Change 1983-2003: -0.2082
Trend: Elasticity is decreasing over time


### Interpretation Questions:

**a)** In your own words, what is the qualitative message from these regressions about the elasticity of VKT with respect to lane km?

**Answer:** For a 1% increase in roads, there is an increase in VKT of less than 1%. The elasticity is positive but inelastic (less than 1), indicating that while more highway capacity does lead to more vehicle travel, the response is proportionally smaller than the increase in capacity.

**b)** What can we learn from here about the change in this relationship over time?

**Answer:** This elasticity has been decreasing over time. From 1983 to 2003, the VKT elasticity with respect to highway kilometers has declined, suggesting that the induced demand effect of highway expansion has weakened over the decades.

## Question 2.3: Table 5 - First-Difference Analysis

**Task:** Replicate column 2 in panel A of Table 5, but only use data from 1993-2003 in the regression.

**Regression specification:**
```
Δlog(vmt) = β₀ + β₁*Δlog(ln_km) + β₂*Δlog(pop) + ε
```
where Δ represents the change from 1993 to 2003.

**Note:** Use heteroskedasticity-robust standard errors (HC1 type).

In [13]:
# Create first-difference variables (1993-2003 changes)
# Equivalent to R code:
# dtDataUse$dVMT9303 <- log(dtDataUse$vmt_IH_03)-log(dtDataUse$vmt_IH_93)
# dtDataUse$dLnKM9303 <- log(dtDataUse$ln_km_IH_03)-log(dtDataUse$ln_km_IH_93)
# dtDataUse$dpop9303 <- log(dtDataUse$pop00)-log(dtDataUse$pop90)

dtDataUse['dVMT9303'] = dtDataUse['log_vmt_03'] - dtDataUse['log_vmt_93']
dtDataUse['dLnKM9303'] = dtDataUse['log_km_03'] - dtDataUse['log_km_93']
dtDataUse['dpop9303'] = dtDataUse['log_pop00'] - dtDataUse['log_pop90']

print("First-difference variables created (1993-2003 changes):")
print("="*55)
print(f"dVMT9303 (Δlog VMT):     Mean = {dtDataUse['dVMT9303'].mean():.4f}, Std = {dtDataUse['dVMT9303'].std():.4f}")
print(f"dLnKM9303 (Δlog km):     Mean = {dtDataUse['dLnKM9303'].mean():.4f}, Std = {dtDataUse['dLnKM9303'].std():.4f}")
print(f"dpop9303 (Δlog pop):     Mean = {dtDataUse['dpop9303'].mean():.4f}, Std = {dtDataUse['dpop9303'].std():.4f}")

# Show summary statistics for difference variables
print("\nSummary of first-difference variables:")
print(dtDataUse[['dVMT9303', 'dLnKM9303', 'dpop9303']].describe())

First-difference variables created (1993-2003 changes):
dVMT9303 (Δlog VMT):     Mean = 0.3277, Std = 0.2011
dLnKM9303 (Δlog km):     Mean = 0.0469, Std = 0.1925
dpop9303 (Δlog pop):     Mean = 0.1233, Std = 0.0971

Summary of first-difference variables:
         dVMT9303   dLnKM9303  dpop9303
count  228.000000  228.000000     228.0
mean     0.327702    0.046881  0.123295
std      0.201066    0.192464  0.097145
min     -0.479029   -0.836369 -0.056858
25%      0.229917   -0.016680  0.050479
50%      0.310383    0.039838  0.118889
75%      0.426359    0.124136   0.17702
max      1.369009    1.150990  0.606092


In [15]:
# First-difference regression: dVMT9303 ~ dLnKM9303 + dpop9303
# Equivalent to R: dif9303 <- lm(dVMT9303 ~ dLnKM9303 + dpop9303, data = dtDataUse)
dif9303 = smf.ols('dVMT9303 ~ dLnKM9303 + dpop9303', data=dtDataUse).fit()

# Get robust standard errors using get_robustcov_results() method
dif9303_robust = dif9303.get_robustcov_results(cov_type='HC1')

print("First-Difference Regression Results (1993-2003):")
print("="*60)
print("Regression: Δlog(VMT) ~ Δlog(highway_km) + Δlog(population)")
print("="*60)
print(f"{'Variable':<15} {'Coefficient':<12} {'Robust SE':<12} {'t-stat':<12}")
print("-" * 60)

# Use the original parameter names and access robust results by index
for i, param_name in enumerate(dif9303.params.index):
    coef = dif9303_robust.params[i]
    se = dif9303_robust.bse[i]
    t_stat = dif9303_robust.tvalues[i]
    print(f"{param_name:<15} {coef:>11.4f} {se:>11.4f} {t_stat:>11.4f}")

print(f"\nR-squared: {dif9303_robust.rsquared:.4f}")
print(f"Observations: {dif9303_robust.nobs}")

# Key coefficient (elasticity from first-difference model)
elasticity_fd = dif9303_robust.params[1]  # Second parameter (dLnKM9303)
print(f"\n*** First-Difference VKT elasticity w.r.t. highway km: {elasticity_fd:.4f} ***")

print(f"\n{'='*60}")
print("COMPARISON WITH PAPER RESULTS:")
print(f"{'='*60}")
print(f"Our estimate (1993-2003): {elasticity_fd:.2f}")
print(f"Paper estimate:           1.05")
print(f"Difference:               {elasticity_fd - 1.05:.2f}")
print("\nNote: This difference suggests the elasticity has been")
print("decreasing over time, consistent with the cross-sectional results.")

First-Difference Regression Results (1993-2003):
Regression: Δlog(VMT) ~ Δlog(highway_km) + Δlog(population)
Variable        Coefficient  Robust SE    t-stat      
------------------------------------------------------------
Intercept            0.2536      0.0125     20.2251
dLnKM9303            0.8574      0.0624     13.7361
dpop9303             0.2754      0.0860      3.2019

R-squared: 0.6788
Observations: 228.0

*** First-Difference VKT elasticity w.r.t. highway km: 0.8574 ***

COMPARISON WITH PAPER RESULTS:
Our estimate (1993-2003): 0.86
Paper estimate:           1.05
Difference:               -0.19

Note: This difference suggests the elasticity has been
decreasing over time, consistent with the cross-sectional results.


### Interpretation Questions:

**a)** What is your point estimate for the elasticity?

**Answer:** 0.86 (approximately - this will be the actual value when you run the code)

**b)** How does this differ from the one reported in the paper (1.05)?

**Answer:** The estimate in the paper is higher at 1.05. Our estimate using 1993-2003 data is lower.

**c)** Can we learn anything from this difference?

**Answer:** We learn that the elasticity has been decreasing over time (also reflected in the paper's findings). This suggests that the fundamental law of road congestion - where increased road capacity leads to proportional increases in traffic - may be weakening over time, possibly due to changing urban form, transportation preferences, or other structural changes in the economy.

## Additional Analysis (Optional)

Use this space for any additional analysis or robustness checks you'd like to explore.

In [ ]:
# Optional: Additional analysis


## References

Duranton, G., & Turner, M. A. (2011). The fundamental law of road congestion: Evidence from US cities. *American Economic Review*, 101(6), 2616-2652.